In [12]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.models import Model, load_model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout,GlobalAveragePooling1D,Conv1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 20000
maxlen = 100


train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").values
print(y.shape)

(95851, 6)


In [13]:
print('test len',len(test))

test len 226998


In [14]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

(95851, 100) (226998, 100)


In [15]:
from sklearn.metrics import log_loss,accuracy_score

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

In [16]:
def get_cnn_model():
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Conv1D(64,
             3,
             padding='valid',
             activation='relu',
             strides=1)(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(32, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
print('def model done')

def model done


In [17]:
from sklearn.model_selection import KFold
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((95851,6)),np.zeros((226998,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = get_cnn_model()
        batch_size = 64
        epochs = 5
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
        print('temp eval')
        eval_val(hold_out_y,hold_out_pred)
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    print('all eval',eval_val(y,train_pred))
    return train_pred, test_pred


train_pred,test_pred = kf_train()
print(train_pred.shape,test_pred.shape)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/cnn_1_csv.gz", index=False, compression='gzip')
import pickle
with open('../features/cnn_1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
print('done')

Train on 63900 samples, validate on 31951 samples
Epoch 1/5
63900/63900 [==============================] - 11s 165us/step - loss: 0.0914 - acc: 0.9718 - val_loss: 0.0536 - val_acc: 0.9809
Epoch 2/5
63900/63900 [==============================] - 10s 149us/step - loss: 0.0498 - acc: 0.9817 - val_loss: 0.0522 - val_acc: 0.9813
Epoch 3/5
63900/63900 [==============================] - 10s 149us/step - loss: 0.0415 - acc: 0.9841 - val_loss: 0.0541 - val_acc: 0.9806
Epoch 4/5
63900/63900 [==============================] - 10s 149us/step - loss: 0.0353 - acc: 0.9863 - val_loss: 0.0563 - val_acc: 0.9807
Epoch 5/5
63900/63900 [==============================] - 10s 149us/step - loss: 0.0298 - acc: 0.9884 - val_loss: 0.0626 - val_acc: 0.9803
temp eval
0 0.11644498065 0.957247034522
1 0.0227608195935 0.990548026666
2 0.0554785950244 0.980063221808
3 0.016691769332 0.996432036556
4 0.0700030086583 0.971894463397
5 0.0315602353003 0.991612156114
final 0.0521565680931 0.981299489844
Train on 63901 sam

In [18]:
train_pred[:10].round(3)

array([[ 0.22 ,  0.001,  0.017,  0.002,  0.038,  0.006],
       [ 0.006,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.003,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.003,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.004,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.041,  0.   ,  0.006,  0.001,  0.005,  0.003],
       [ 0.026,  0.   ,  0.002,  0.   ,  0.003,  0.001],
       [ 0.052,  0.   ,  0.002,  0.   ,  0.007,  0.002],
       [ 0.018,  0.   ,  0.002,  0.   ,  0.002,  0.001],
       [ 0.03 ,  0.   ,  0.002,  0.   ,  0.003,  0.002]])

In [19]:
y[:10]

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [20]:
def get_nn_model():
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
print('def model done')

def model done


In [21]:
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((95851,6)),np.zeros((226998,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = get_nn_model()
        batch_size = 64
        epochs = 6
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, 
                                     save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
        print('temp eval')
        eval_val(hold_out_y,hold_out_pred)
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    print('all eval',eval_val(y,train_pred))
    return train_pred, test_pred


train_pred,test_pred = kf_train()
print(train_pred.shape,test_pred.shape)

Train on 63900 samples, validate on 31951 samples
Epoch 1/6
63900/63900 [==============================] - 10s 149us/step - loss: 0.1425 - acc: 0.9582 - val_loss: 0.1010 - val_acc: 0.9670
Epoch 2/6
63900/63900 [==============================] - 9s 140us/step - loss: 0.0707 - acc: 0.9765 - val_loss: 0.0570 - val_acc: 0.9802
Epoch 3/6
63900/63900 [==============================] - 9s 138us/step - loss: 0.0527 - acc: 0.9811 - val_loss: 0.0535 - val_acc: 0.9811
Epoch 4/6
63900/63900 [==============================] - 9s 137us/step - loss: 0.0459 - acc: 0.9828 - val_loss: 0.0521 - val_acc: 0.9813
Epoch 5/6
63900/63900 [==============================] - 8s 132us/step - loss: 0.0417 - acc: 0.9841 - val_loss: 0.0524 - val_acc: 0.9815
Epoch 6/6
63900/63900 [==============================] - 8s 132us/step - loss: 0.0380 - acc: 0.9852 - val_loss: 0.0531 - val_acc: 0.9816
temp eval
0 0.114027832404 0.95849895152
1 0.0236239090643 0.990861005915
2 0.0557343679851 0.978842602735
3 0.0174336698526 0.

In [22]:
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/nn_1_csv.gz", index=False, compression='gzip')
import pickle
with open('../features/nn_1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
print('done')

done
